In [1]:
from pathlib import Path
import subprocess
import os

import pymupdf
from pymupdf import TextWriter

In [2]:
XOJ_PATH = Path('../tests/test_1.xoj')
PDF_PATH = XOJ_PATH.with_suffix('.pdf')
PDF_PATH_OCR = PDF_PATH.parent / Path(PDF_PATH.stem+'_ocrd').with_suffix('.pdf')

First, turn test file from `xoj` into `pdf`.

In [3]:
subprocess.run(f'xournalpp {XOJ_PATH} -p {PDF_PATH}', shell=True)


** Message: 14:12:32.084: PDF file successfully created


CompletedProcess(args='xournalpp ../tests/test_1.xoj -p ../tests/test_1.pdf', returncode=0)

Next, using some dummy text and position, try to add hidden (=searchable) text to PDF. Based on [this issue](https://github.com/pymupdf/PyMuPDF/discussions/775#discussioncomment-219578) found using G"python add hidden text to pdf".

In [4]:
try:
    os.remove(PDF_PATH_OCR)
except Exception as e:
    print(f'Error encountered while deleting {PDF_PATH_OCR}', e)

doc = pymupdf.open(PDF_PATH)

for page in doc:

    text = 'a wee test!'

    tw = TextWriter(page.rect)
    tw.append(
        (0.0, 0.0),
        text,
        font=None,
        fontsize=11,
    )
    tw.write_text(page, render_mode=3) # render_mode=3 == hidden text
    # TODO: How to adjust text to a given bounding box as to perfectly align
    #       with text recognition?

doc.ez_save(PDF_PATH_OCR)

[Errno 2] No such file or directory: '../tests/test_1_ocrd.pdf'


In [5]:
# TODO: I want to build prediction code that can run both in a CLI and in a notebook like this here. Also, I'd like to be able to set the model flexibly.